## Template

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from decouple import config
from fastembed import (
    TextEmbedding, 
    SparseTextEmbedding, 
    LateInteractionTextEmbedding
)
from pandas import DataFrame
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    Fusion,
    VectorParams, 
    SparseVectorParams,
    SparseIndexParams,
    MultiVectorConfig,
    MultiVectorComparator,
    Modifier,
    HnswConfigDiff
)

from rag.models import (
    DenseModelConfig, 
    SparseModelConfig,
    RerankingModelConfig, 
)
from rag.repositories import (
    DenseSearchRepository,
    SparseSearchRepository,
    HybridFusionSearchRepository, 
    HybridRerankingSearchRepository
)
from rag.utils import load_datasets, Evaluator


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')
NVIDIA_API_KEY = config('NVIDIA_API_KEY')
CUDA=config('CUDA', cast=bool)
PROVIDER = 'CUDAExecutionProvider' if CUDA else 'CPUExecutionProvider'

### Client

In [2]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='dense_collection'), CollectionDescription(name='fusion_collection'), CollectionDescription(name='sparse_collection2'), CollectionDescription(name='reranking_collection'), CollectionDescription(name='sparse_collection')]


### Models

In [3]:
dense_model_name = 'BAAI/bge-small-en-v1.5'
dense_model = TextEmbedding(
    model_name=dense_model_name,
    providers=[PROVIDER]
)

sparse_model_name = 'Qdrant/bm25'
sparse_model = SparseTextEmbedding(
    model_name=sparse_model_name,
    providers=[PROVIDER],
    k1=1.5,
    b=0.75
)

reranking_model_name = 'colbert-ir/colbertv2.0'
reranking_model = LateInteractionTextEmbedding(
    model_name=reranking_model_name,
    providers=[PROVIDER]
)

dense_model_config = DenseModelConfig(
    name=dense_model_name,
    vector_params=VectorParams(
        size=384,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=16,
            ef_construct=128,
            on_disk=True
        ),
        on_disk=True
    )
)

sparse_model_config = SparseModelConfig(
    name=sparse_model_name,
    sparse_vector_params=SparseVectorParams(
        index=SparseIndexParams(
            on_disk=True
        ),
        modifier=Modifier.IDF
    )
)

reranking_model_config = RerankingModelConfig(
    name=reranking_model_name,
    vector_params=VectorParams(
        size=128,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=0     # disable HNSW
        ),
        on_disk=True,
        multivector_config=MultiVectorConfig(
            comparator=MultiVectorComparator.MAX_SIM,
        )
    )
)

c:\Users\lukap\AppData\Local\pypoetry\Cache\virtualenvs\rag-LTDKTwC4-py3.12\Lib\site-packages\fastembed\common\onnx_model.py:96: RuntimeWarning: Attempt to set CUDAExecutionProvider failed. Current providers: ['CPUExecutionProvider'].If you are using CUDA 12.x, install onnxruntime-gpu via `pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/`
  warnings.warn(


### Documents

In [4]:
corpus_df, queries_df, qrels_df = load_datasets(2_000, 2_000)

print(len(corpus_df), len(queries_df), len(qrels_df))

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

27 27 28


### Optimization

#### Init

In [5]:
COLLECTION_NAME = 'custom_collection'

search_repository = DenseSearchRepository(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config
)
evaluator = Evaluator(dfs=(corpus_df, queries_df, qrels_df))

#### Setup

In [7]:
evaluator.setup(
    collection_name=COLLECTION_NAME,
    repository=search_repository,
    dense_model=dense_model,
    sparse_model=None,
    reranking_model=None
)

#### Run

In [8]:
top_k = 5
metrics = [
    'mrr',
    'map',
    f'precision@{top_k}',
    f'recall@{top_k}',
    f'ndcg@{top_k}'
]
results = evaluator.run(
    metrics=metrics,
    top_k=top_k,
    scale_k=10,
    fusion_algorithm=Fusion.RRF
)
results_df = DataFrame([results])
results_df

,mrr,map,precision@5,recall@5,ndcg@5
0,0.981481,0.981481,0.207407,1.0,0.986331


#### Clear

In [9]:
evaluator.clear()

True

### Plots

In [ ]:
# TODO
# create plots
# export plots in .svg format